In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d moeinkpr/snappfood-comments

Dataset URL: https://www.kaggle.com/datasets/moeinkpr/snappfood-comments
License(s): MIT
  0% 0.00/98.1M [00:00<?, ?B/s]
100% 98.1M/98.1M [00:00<00:00, 1.32GB/s]


In [3]:
!unzip snappfood-comments.zip

Archive:  snappfood-comments.zip
  inflating: comments.csv            
  inflating: vendors.csv             


In [4]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 603, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 603 (delta 131), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (603/603), 199.38 KiB | 1.53 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Installing RAPIDS remaining 25.08 libraries
Using Python 3.12.11 environment at: /usr
Resolved 180 packages in 1.94s
Prepared 41 packages in 48.38s
Uninstalled 31 packages in 1.31s
Installed 41 packages in 375ms
 - bokeh==3.7.3
 + bokeh==3.6.3
 + cucim-cu12==25.8.0
 + cuda-bindings==12.9.2
 + cuda-pathfinder==1.3.0
 - cuda-python==12.6.2.post1
 + cuda-python==12.9.2
 - cudf-cu12==25.6.0 (from https://pypi.nvidia.com/cudf-cu12/cudf_cu12-25.6.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl)
 + cudf-cu12==25.8.0
 + cugraph-cu12==25.8.0
 - cuml-cu12==25.6.0
 + cuml-cu12==25.8.0
 - cuvs-cu12==25.6.1
 + cuvs-cu12==25.8.0
 + cu

In [5]:
import cudf

In [6]:
comments_table=cudf.read_csv("comments.csv")
vendors_table=cudf.read_csv("vendors.csv")

comments_table_copy=comments_table.drop(columns=["commentId","date","sender","rating","customerId","feeling","status","foods","replies"])
comments_table_copy
vendors_table_copy=vendors_table.drop(columns=["id","highlight","description","address","rating","title"])
vendors_table_copy

,code,commentCount,vendorType
0,pzlkj3,24112,RESTAURANT
1,0lvonp,10988,RESTAURANT
2,3drlyx,5815,RESTAURANT
3,klw59g,12,RESTAURANT
4,32m1rl,13542,RESTAURANT
...,...,...,...
2250,3x25xw,7391,RESTAURANT
2251,0rel4o,46,RESTAURANT
2252,0y16yg,22278,RESTAURANT
2253,znv94e,66,RESTAURANT


In [19]:
merged_df = comments_table_copy.merge(vendors_table_copy,on="code",how="left")
merged_df = merged_df[merged_df["vendorType"] == "RESTAURANT"]

merged_df=merged_df.drop(columns=["vendorType"])
merged_df["deliveryComment"].fillna("",inplace=True)


In [20]:
merged_df['text_raw'] = (merged_df['commentText'].fillna('') + ' ' + merged_df['deliveryComment'].fillna(''))
merged_df = merged_df.drop(columns=['commentText', 'deliveryComment'])
